In [17]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

In [2]:
model = tf.keras.Sequential()

# model.add(tfa.layers.EmbeddingBag(input_dim=1000, output_dim=64, input_shape=[32]))
model.add(layers.Embedding(input_dim=1000, output_dim=64, input_shape=[32]))
model.add(layers.Flatten())

model.add(layers.Dense(64, activation="tanh"))
                      #  kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
                      # bias_regularizer=regularizers.L2(1e-4),
                      # activity_regularizer=regularizers.L2(1e-5)))
model.add(layers.Dense(64,activation="tanh"))
                      #  kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
                      # bias_regularizer=regularizers.L2(1e-4),
                      # activity_regularizer=regularizers.L2(1e-5)))
model.add(layers.Dense(32,activation="tanh"))
                      #  kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),
                      # bias_regularizer=regularizers.L2(1e-4),
                      # activity_regularizer=regularizers.L2(1e-5)))

model.add(layers.Dense(1, activation="sigmoid"))

model.compile(
    loss = "binary_crossentropy",
    optimizer = "RMSprop",
    metrics = ["accuracy"]
)

model.summary()

2022-06-06 14:08:57.660390: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-06 14:08:57.660445: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-06 14:08:57.660495: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (uwu): /proc/driver/nvidia/version does not exist
2022-06-06 14:08:57.713422: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 64)            64000     
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 64)                131136    
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 201,409
Trainable params: 201,409
Non-trai

In [10]:
teks = tf.constant(["i love you",
               "When using this layer as the first layer in a model, provide an input_shape argument (tuple of integers or None, e.g. (10, 128) for sequences of 10 vectors of 128-dimensional vectors, or (None, 128) for variable-length sequences of 128-dimensional vectors.",
               "Indonesia, dengan nama resmi Republik Indonesia atau lengkapnya Negara Kesatuan Republik Indonesia, adalah sebuah negara kepulauan di Asia Tenggara yang dilintasi garis khatulistiwa dan berada di antara daratan benua Asia dan Oseania, serta antara Samudra Pasifik dan Samudra Hindia",
               "hi kami dari red team",
               "hi we are from red team",
               "mantab pak eko",
               "lu cakep juga yak"])

interpreter = tf.lite.Interpreter(model_path="/home/vannyezha/project/IOH-chat-app/MachineLearning/code/binary_language_classification/model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# print(input_details)
# print(output_details)

# must have 2 dimension and float32 dtype


ValueError: Could not open '/home/vannyezha/project/IOH-chat-app/MachineLearning/code/binary_language_classification/model.tflite'.

In [15]:
import json

with open("/home/vannyezha/project/IOH-chat-app/MachineLearning/code/binary_language_classification/model/word_index.json") as f:
  kamus = json.load(f)

teks = tf.constant(["i love you",
               "When using this layer as the first layer in a model, provide an input_shape argument (tuple of integers or None, e.g. (10, 128) for sequences of 10 vectors of 128-dimensional vectors, or (None, 128) for variable-length sequences of 128-dimensional vectors.",
               "Indonesia, dengan nama resmi Republik Indonesia atau lengkapnya Negara Kesatuan Republik Indonesia, adalah sebuah negara kepulauan di Asia Tenggara yang dilintasi garis khatulistiwa dan berada di antara daratan benua Asia dan Oseania, serta antara Samudra Pasifik dan Samudra Hindia",
               "hi kami dari red team",
               "hi we are from red team",
               "mantab pak eko",
               "lu cakep juga yak"])

def tf_lower_and_split_punct(text):
  text = tf.strings.lower(text)
  text = tf.strings.regex_replace(text, "[!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]", '')

  text = tf.strings.strip(text)
  text = tf.strings.split(text)
  return text

def sequenize(teks):
  hasil = list()
  counter = 32
  for i in teks:
    if i.numpy().decode() in kamus.keys():
      hasil.append(kamus[i.numpy().decode()])
      counter -= 1
      if counter == 0:
        return np.array([hasil], dtype=np.float32)
    else:
      hasil.append(1)
      counter -= 1
      if counter == 0:
        return np.array([hasil], dtype=np.float32)

  for i in range(counter):
    hasil.append(0)

  return tf.constant([hasil])

In [18]:
interpreter = tf.lite.Interpreter(model_path="/home/vannyezha/project/IOH-chat-app/MachineLearning/code/binary_language_classification/model")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


# must have 2 dimension and float32 dtype
for i in teks:
  input_data = np.array(sequenize(tf_lower_and_split_punct(i)), dtype=np.float32)
  interpreter.set_tensor(input_details[0]['index'], input_data)
  interpreter.invoke()

  predictions = interpreter.get_tensor(output_details[0]['index'])

  print(i.numpy().decode())
  print(predictions)

ValueError: Mmap of '119' at offset '0' failed with error '19'.